In [28]:
# !pip install pyowm
import sounddevice as sd
import pyttsx3
import speech_recognition as sr
import numpy as np
import io
from io import BytesIO
import wave
from pyowm import OWM
from pyowm.utils import config
import requests
from pyowm.utils import timestamps

In [37]:
api_key = 'ecd187f849412526e573fd50f5b2173f'
owm = OWM(api_key)  # Substitua pela sua chave API do OpenWeatherMap
mgr = owm.weather_manager()
location = 'Salto, BR'

observation = mgr.weather_at_place(location)  # Substitua pela localização desejada
weather = observation.weather

In [40]:
url = f'http://api.openweathermap.org/data/2.5/forecast?q={location}&appid={api_key}&units=metric'

response = requests.get(url)
data = response.json()

for forecast in data['list']:
    date = forecast['dt_txt']
    temperature = forecast['main']['temp']
    min_temperature = forecast['main']['temp_min']
    max_temperature = forecast['main']['temp_max']
    rain_probability = forecast.get('rain', {}).get('3h', 0)  # '3h' para probabilidade de chuva nas últimas 3 horas

    print(f"Data e Hora: {date}")
    print(f"Temperatura: {temperature}°C")
    print(f"Temperatura mínima: {min_temperature}°C")
    print(f"Temperatura máxima: {max_temperature}°C")
    print(f"Probabilidade de chuva: {rain_probability} mm")
    print("---")

Data e Hora: 2024-09-17 15:00:00
Temperatura: 22.62°C
Temperatura mínima: 22.62°C
Temperatura máxima: 25.95°C
Probabilidade de chuva: 0 mm
---
Data e Hora: 2024-09-17 18:00:00
Temperatura: 23.52°C
Temperatura mínima: 23.52°C
Temperatura máxima: 25.33°C
Probabilidade de chuva: 0 mm
---
Data e Hora: 2024-09-17 21:00:00
Temperatura: 20.18°C
Temperatura mínima: 18.96°C
Temperatura máxima: 20.18°C
Probabilidade de chuva: 0 mm
---
Data e Hora: 2024-09-18 00:00:00
Temperatura: 15.83°C
Temperatura mínima: 15.83°C
Temperatura máxima: 15.83°C
Probabilidade de chuva: 0 mm
---
Data e Hora: 2024-09-18 03:00:00
Temperatura: 15.8°C
Temperatura mínima: 15.8°C
Temperatura máxima: 15.8°C
Probabilidade de chuva: 0 mm
---
Data e Hora: 2024-09-18 06:00:00
Temperatura: 14.91°C
Temperatura mínima: 14.91°C
Temperatura máxima: 14.91°C
Probabilidade de chuva: 0 mm
---
Data e Hora: 2024-09-18 09:00:00
Temperatura: 13.67°C
Temperatura mínima: 13.67°C
Temperatura máxima: 13.67°C
Probabilidade de chuva: 0 mm
---
Da

In [45]:
def falar(texto):
    engine = pyttsx3.init()
    engine.say(texto)
    engine.runAndWait()

def capturar_audio():
    samplerate = 44100
    duration = 5  # Duração de 5 segundos
    print("Ouvindo...")
    audio = sd.rec(int(samplerate * duration), samplerate=samplerate, channels=1, dtype='int16')
    sd.wait()
    return audio, samplerate
    
def probabilidade_chuva_hoje():
    rain_probability = weather.rain.get('12h', 0)

    falar(f"Probabilidade de chuva: {rain_probability} mm")

def temperatura_semana():
    for forecast in data['list']:
        date = forecast['dt_txt']
        temperature = forecast['main']['temp']
        min_temperature = forecast['main']['temp_min']
        max_temperature = forecast['main']['temp_max']

        falar('Aqui esta o seu resultado')
        print(f"Data e Hora: {date}")
        print(f"Temperatura: {temperature}°C")
        print(f"Temperatura mínima: {min_temperature}°C")
        print(f"Temperatura máxima: {max_temperature}°C")
        print("---")

def probabilidade_chuva_semana():
    for forecast in data['list']:
        date = forecast['dt_txt']
        rain_probability = forecast.get('rain', {}).get('3h', 0)  # '3h' para probabilidade de chuva nas últimas 3 horas

        falar('Aqui esta o seu resultado')
        print(f"Data e Hora: {date}")
        print(f"Probabilidade de chuva: {rain_probability} mm")
        print("---")

def temperatura_hoje():
    temperature = weather.temperature('celsius')['temp']
    min_temperature = weather.temperature('celsius')['temp_min']
    max_temperature = weather.temperature('celsius')['temp_max']

    falar(f"Temperatura atual: {temperature}°C")
    falar(f"Temperatura mínima: {min_temperature}°C")
    falar(f"Temperatura máxima: {max_temperature}°C")
    
def audio_para_speech_recognition(audio, samplerate):
    # Converte o áudio capturado em um formato compatível com o speech_recognition
    audio_wav = io.BytesIO()
    wav_file = wave.open(audio_wav, "wb")
    wav_file.setnchannels(1)
    wav_file.setsampwidth(2)
    wav_file.setframerate(samplerate)
    wav_file.writeframes(audio)
    wav_file.close()
    audio_wav.seek(0)
    
    return sr.AudioFile(audio_wav)

def reconhecer_fala():
    reconhecedor = sr.Recognizer()

    # Captura o áudio usando sounddevice
    audio, samplerate = capturar_audio()

    # Converte o áudio para o formato aceito pelo speech_recognition
    audio_wav = audio_para_speech_recognition(audio, samplerate)

    with audio_wav as source:
        try:
            audio_data = reconhecedor.record(source)
            comando = reconhecedor.recognize_google(audio_data, language='pt-BR').lower()
            falar(f"Você disse: {comando}")
            return comando
        except sr.UnknownValueError:
            falar("Não entendi o que foi dito.")
        except sr.RequestError as e:
            falar(f"Erro ao solicitar resultados do serviço de reconhecimento: {e}")
        return None

def processar_pergunta(pergunta):
    pergunta_normalizada = pergunta.lower().strip()
    print(f"Pergunta recebida: {pergunta_normalizada}")

    if "temperatura de hoje" in pergunta_normalizada:
        temperatura_hoje()
    elif "temperatura da semana" in pergunta_normalizada:
        temperatura_semana()
    elif "probabilidade de chuva hoje" in pergunta_normalizada:
        probabilidade_chuva_hoje()
    elif "probabilidade de chuva essa semana" in pergunta_normalizada:
        probabilidade_chuva_semana()
    else:
        falar('Não entendi')
        
def perguntas():
    perguntas = [
        "Temperatura de hoje",
        "Temperatura da semana",
        "Probabilidade de chuva hoje",
        "Probabilidade de chuva essa semana",
    ]
    lista_perguntas = "\n".join(f"{i + 1}. {pergunta}" for i, pergunta in enumerate(perguntas))
    falar("Aqui estão as perguntas que você pode fazer: ")
    print("Lista de perguntas:")
    print(lista_perguntas)

def Assistente():
    perguntas()
    pergunta = reconhecer_fala()
    if pergunta:
        processar_pergunta(pergunta)

In [46]:
Assistente()

Lista de perguntas:
1. Temperatura de hoje
2. Temperatura da semana
3. Probabilidade de chuva hoje
4. Probabilidade de chuva essa semana
Ouvindo...
